<a href="https://colab.research.google.com/github/paritachavda/Face-Recognition-using-Sparse-Classifier/blob/main/Yale_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sparse representation based classifier on Extended Yale B dataset**
*   Dataset Info: It includes 31 persons images under 64 lightning situations.(Total Images: 1984)
*   Training:32 images per person(992 images)
*  Testing: 32 images per person(992 images)
*  Original dimension:1024
*  Reduced dimension: 140
*   Accuracy: 97.37%








In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
from PIL import Image
import os, sys

In [ ]:
def read_images(path, sz=None,sz0=168,sz1=192):     
    c=0
    X,y = [], []
    for dirname , dirnames , filenames in os.walk(path):
        # print(dirname)
        # print(dirnames)
        # print(filenames)
        for subdirname in dirnames:
            subject_path = os.path.join(dirname , subdirname) 
            # count=0
            for filename in os.listdir(subject_path):
                try:
                    # count=count+1
                    im = Image.open(os.path.join(subject_path , filename)) 
                    # im = im.convert("L")
                    
                    if ((im.size[0] == sz0) & (im.size[1]==sz1)):
                        if (sz is not None):
                            im = im.resize(sz, Image.NEAREST)     
                        X.append(np.asarray(im, dtype=np.uint8)) 
                        y.append(c)
                except IOError:
                    pass
                except:
                    print("Unexpected error:", sys.exc_info()[0])
                    raise
            
            c = c+1          
    
    return [X,y]

In [ ]:
!unzip -q '/content/drive/My Drive/thesis/dataset/CroppedYale.zip'

In [ ]:
path_images = '/content/CroppedYale' 
X,y = read_images(path_images, sz=(32,32))
print("Number of loaded items ",len(X))

Number of loaded items  2442


In [ ]:
import collections
my_dict = collections.defaultdict(int)
for i in y:
    my_dict[i] += 1
temp = min(my_dict.items(), key=lambda x: x[1])
print(temp)
min_num_img = temp[1]
# my_dict

(0, 64)


In [ ]:
number_of_classes=31
width=32
height=32
input_data= []
input_label= []
prev=0
for i in range(number_of_classes):
    img_in_class = my_dict[i]
   
    for j in range(min_num_img):
        input_data.append(X[prev+j])
        input_label.append(y[prev+j])
    
    prev += img_in_class

In [ ]:
np.random.seed(2019)

Xtrain, Xtest = [], []
ytrain, ytest = [], []

for i in range(len(input_data)):
    if i%64==0:
        train = np.random.choice(64,32,replace=False)
          
    if ((i%64 in train)):
        Xtrain.append(input_data[i])
        ytrain.append(input_label[i])        
    else:
        Xtest.append(input_data[i])
        ytest.append(input_label[i])      

len(Xtrain)

992

In [ ]:
img_in_class = 32
training_data= np.zeros((number_of_classes*img_in_class,width*height))
training_label= np.zeros(number_of_classes*img_in_class)

for i in range(number_of_classes):
    for c in range(img_in_class):
        training_data[i*img_in_class+c,:] = Xtrain[i*img_in_class+c].flatten()
        training_label[i*img_in_class+c]=ytrain[i*img_in_class+c]    
    
training_data_col = training_data.T


In [ ]:
X_toconcat_test = [np.reshape(e,(Xtest[0].shape[0]*Xtest[0].shape[1],1)) for e in Xtest]
test_data_col = np.concatenate(X_toconcat_test,axis=1) 

In [ ]:
from sklearn.decomposition import PCA

def eigenfaces(X_train,X_test,n_components=200):  
    pca = PCA(n_components=n_components, svd_solver='randomized',
      whiten=True).fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    return X_train_pca, X_test_pca,pca.explained_variance_  

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
Xtrain_std = ss.fit_transform(training_data_col)
Xtest_std = ss.fit_transform(test_data_col)

Xtrain_PCA, Xtest_PCA,eigen_vals = eigenfaces(Xtrain_std.T,Xtest_std.T,n_components=200)


In [ ]:
for inx in range(len(Xtrain_PCA)):
  Xtrain_PCA[inx] = np.divide(Xtrain_PCA[inx],eigen_vals) 


In [ ]:
Xtrain_PCA[0][0:5]

array([-0.00328853, -0.00371869, -0.01427543,  0.00080974,  0.01914916])

In [ ]:
Xtest_PCA[0][0:5]

array([ 0.11433962,  0.95263196,  1.64479569,  0.84129758, -0.3050487 ])

In [ ]:
for inx in range(len(Xtest_PCA)):
  Xtest_PCA[inx] = np.divide(Xtest_PCA[inx],eigen_vals)

In [ ]:
Xtest_PCA[0][0:5]

array([ 0.0003052 ,  0.00736459,  0.02457456,  0.03145967, -0.01499844])

In [ ]:
Xtrain_col, Xtest_col = Xtrain_PCA.T, Xtest_PCA.T

In [ ]:
np.random.seed(2019)
from numpy.random import rand

alpha = 0.010
pca_components = 200
number_chosen_components=140

p_zero = np.random.rand(pca_components,number_chosen_components)
y_tr = p_zero.transpose().dot(Xtrain_col)

In [ ]:
y_tr

array([[ -4.52525692, -26.44185408, -25.4680328 , ...,  10.58300278,
          0.06558259, -28.26233276],
       [  8.22579566, -14.65624994, -30.25045171, ...,  11.48292953,
         -0.17850183, -12.76153595],
       [ 17.54996849, -12.48052803, -30.82179526, ..., -15.51810738,
         16.5076912 , -12.88308979],
       ...,
       [-23.62641656, -21.21125994,  -7.43357762, ...,  25.83588901,
         -4.30968738, -30.38447277],
       [ -9.67587994, -22.78345989, -28.79139715, ...,  -2.73572279,
         -5.82590591, -30.3280251 ],
       [  0.39639443,  15.42527845, -32.14471451, ...,   7.47225536,
        -12.96906778, -15.54882434]])

In [ ]:
print(p_zero)

[[0.90348221 0.39308051 0.62396996 ... 0.94282962 0.20161853 0.32548516]
 [0.91542946 0.80026745 0.90502811 ... 0.43501387 0.87555035 0.55869017]
 [0.23663512 0.06061889 0.77485038 ... 0.42688347 0.69432765 0.76174903]
 ...
 [0.22828824 0.69057527 0.21124483 ... 0.32022098 0.00974336 0.89718423]
 [0.86959116 0.42671688 0.67344954 ... 0.21846711 0.73290182 0.22027651]
 [0.66657454 0.27559988 0.52202391 ... 0.51201401 0.47320117 0.7738579 ]]


In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Lasso
from tqdm import tqdm

sp_re = np.zeros(shape=(y_tr.shape[1]-1,y_tr.shape[1]))
for i in tqdm(range(y_tr.shape[1])):  
    clf = Lasso(alpha)
    y = y_tr[:,i]
    y_tr = np.delete(y_tr,i,1)
    clf.fit(y_tr,y)
    y_tr=np.insert(y_tr,i,y,axis=1)
    sp_re[:,i] = clf.coef_


100%|██████████| 992/992 [03:01<00:00,  5.47it/s]


In [ ]:
sp_re

array([[ 0.11567302,  0.26448786,  0.        , ...,  0.05890141,
         0.        , -0.        ],
       [ 0.        , -0.21476293, -0.05077747, ...,  0.00155595,
         0.00307561,  0.        ],
       [-0.        ,  0.06497198,  0.05134005, ...,  0.12692718,
        -0.07465107,  0.13519844],
       ...,
       [ 0.00207912,  0.        ,  0.00028611, ...,  0.06489935,
        -0.02664633, -0.        ],
       [-0.        ,  0.        , -0.        , ...,  0.        ,
        -0.        , -0.        ],
       [-0.        ,  0.03347367,  0.01826911, ..., -0.02359766,
         0.        , -0.        ]])

In [ ]:
sp_re

array([[ 1.69315947e-02,  1.28260475e-01,  1.67982293e-02, ...,
         4.05959681e-02, -3.96801627e-03, -0.00000000e+00],
       [-0.00000000e+00,  3.71003660e-02,  1.99697731e-02, ...,
         2.94315584e-03,  2.01807743e-03,  0.00000000e+00],
       [-0.00000000e+00,  1.17449756e-01, -0.00000000e+00, ...,
         3.68799032e-02, -0.00000000e+00,  4.09484620e-02],
       ...,
       [ 0.00000000e+00,  3.22674923e-06,  9.29582271e-04, ...,
         3.18901665e-02, -3.01880520e-03, -1.28086804e-04],
       [-0.00000000e+00, -0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.75049692e-03, ...,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00]])

In [ ]:
mew = np.zeros((number_of_classes, pca_components))
M = np.zeros((1,pca_components))

for i in range(number_of_classes):
    xa = Xtrain_PCA[img_in_class*i:img_in_class*i+img_in_class,:]
    for j in xa:
        mew[i,:] = np.add(mew[i,:],j)
    mew[i,:] = np.divide(mew[i,:],float(len(xa)))

for i in Xtrain_PCA:
    M = np.add(M,i)
M = np.divide(M,float(Xtrain_PCA.shape[0]))

In [ ]:
normalised_wc_proj_sig = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)

for i in range(number_of_classes):
    for j in range(img_in_class):
        normalised_wc_proj_sig[i*img_in_class+j,:] = np.subtract(Xtrain_PCA[i*img_in_class+j,:],mew[i,:])


sw = np.zeros((pca_components,pca_components))

for i in range(number_of_classes):
    xa = normalised_wc_proj_sig[img_in_class*i:img_in_class*i+img_in_class,:]
    xa = xa.transpose()
    cov = np.dot(xa,xa.T)
    sw = sw + cov

normalised_class = np.ndarray(shape=(number_of_classes, pca_components), dtype=np.float64)
sb = np.zeros((pca_components,pca_components))

for i in range(number_of_classes):
    normalised_class[i,:] = np.subtract(mew[i,:],M)

sb = np.dot(normalised_class.T,normalised_class)
sb = np.multiply(sb,float(img_in_class))

In [ ]:
def delta(x,i,classs):
    
    n,m = len(x),len(classs)
    
    if (n != m):
        print('vectors of differents sizes, cannot operate delta')
        
    tmp = i*np.ones(n)-classs

    for k in range(n):
        if tmp[k]==0:
            tmp[k]=1
        else:
            tmp[k]=0 
            
    return tmp*x

def residu(y,A,x,class_x):
    
    k = np.max(class_x)+1
    r = np.zeros(k)
    
    for i in range(0,k):
        r[i] = np.linalg.norm(y - np.dot(A,delta(x,i,class_x)))
        
    return r

def cost(proj):
    multi_b = np.dot(proj.T,np.dot(sb+Eb,proj))
    numerator = np.trace(multi_b)
    # print(numerator)
    multi_w = np.dot(proj.T,np.dot(Ew+sw,proj))
    denomenator = np.trace(multi_w)
    # print(denomenator)
    final_val = numerator/denomenator
    # print(final_val)
    return final_val

In [ ]:
num_tr = y_tr.shape[1]
Ew = np.zeros((pca_components,pca_components))
Withinclass_error = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in range(num_tr):
    x = Xtrain_col[:,i]
    label = training_label[i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    training_label = np.delete(training_label,i)
    Withinclass_error[i,:] = x-np.dot(Xtrain_col,delta(sp_re[:,i],label,training_label))
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
    training_label=np.insert(training_label,i,label)
    
Ew = np.dot(Withinclass_error.T,Withinclass_error)
Ew = Ew/number_of_classes*img_in_class

In [ ]:
Ew

array([[ 1.81249460e-03, -1.45816752e-04,  5.61042296e-04, ...,
        -3.74628840e-02, -2.88140613e-01,  3.76210488e-01],
       [-1.45816752e-04,  1.86650323e-02, -1.07587634e-03, ...,
        -6.57907114e-01,  2.70789456e-01, -7.59814578e-01],
       [ 5.61042296e-04, -1.07587634e-03,  8.33439252e-02, ...,
         4.87299270e-01, -1.13970206e+00,  1.34721618e+00],
       ...,
       [-3.74628840e-02, -6.57907114e-01,  4.87299270e-01, ...,
         3.83352127e+04, -1.23278383e+03, -2.63230486e+02],
       [-2.88140613e-01,  2.70789456e-01, -1.13970206e+00, ...,
        -1.23278383e+03,  3.81047589e+04, -6.51102805e+02],
       [ 3.76210488e-01, -7.59814578e-01,  1.34721618e+00, ...,
        -2.63230486e+02, -6.51102805e+02,  3.92733070e+04]])

In [ ]:
from tqdm import tqdm
num_data = number_of_classes*img_in_class
Eb = np.zeros((pca_components,pca_components))
classLabels = np.unique(ytrain)
k={}
for label in classLabels:  
    temp = np.where(training_label!=label)
    k.update({label:temp})


In [ ]:
Eb = np.zeros((pca_components,pca_components))
Betweenclass_error = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in tqdm(range(num_tr)):
    tmp = np.zeros((pca_components))
    x = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    label = training_label[i]
    other_label_index=(k[label][0])
    for j in range(other_label_index.shape[0]):
        other_label = training_label[j]
        training_label = np.delete(training_label,j)
        tmp += x-(np.dot(Xtrain_col,delta(sp_re[:,j],other_label,training_label)))
        training_label=np.insert(training_label,j,other_label)
    Betweenclass_error[i,:] = tmp
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
Eb = np.dot(Betweenclass_error.T,Betweenclass_error)
Eb = Eb/(num_data*(number_of_classes-1))

100%|██████████| 992/992 [21:32<00:00,  1.30s/it]


In [ ]:
Eb

array([[ 2.18955884e-01,  2.48086701e-04, -7.55458933e-04, ...,
        -3.23444834e-01, -5.18402824e-01,  2.16935414e-01],
       [ 2.48086701e-04,  1.83679539e+00,  3.87301914e-03, ...,
         1.84402686e-01,  6.33338575e-02, -1.12247184e-01],
       [-7.55458933e-04,  3.87301914e-03,  6.86687646e+00, ...,
         1.00805819e+00, -1.24781577e-01, -1.39183679e+00],
       ...,
       [-3.23444834e-01,  1.84402686e-01,  1.00805819e+00, ...,
         1.47071060e+06, -5.33003360e+02,  6.21866625e+01],
       [-5.18402824e-01,  6.33338575e-02, -1.24781577e-01, ...,
        -5.33003360e+02,  1.49286246e+06, -2.02024134e+02],
       [ 2.16935414e-01, -1.12247184e-01, -1.39183679e+00, ...,
         6.21866625e+01, -2.02024134e+02,  1.58148682e+06]])

In [ ]:
beta = 1
J = np.dot(np.linalg.inv(sw+(beta*Ew)),(sb+(beta*Eb)))
eigenvalues, eigenvectors, = np.linalg.eig(J)

In [ ]:
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]

In [ ]:
print('Number of eigen vectors:',len(eigvalues_sort))
print("k:",number_chosen_components)
proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()
proj_mtx.shape

Number of eigen vectors: 200
k: 140


(200, 140)

In [ ]:
p_zero[0,:5]

array([0.90348221, 0.39308051, 0.62396996, 0.6378774 , 0.88049907])

In [ ]:
proj_mtx[0,:5]

array([-0.17274474, -0.68918031, -0.48925921,  0.82355211,  0.14763142])

In [ ]:
proj_mtx = proj_mtx.real

In [ ]:
cost(proj_mtx)

23.639834878174376

In [ ]:
cost(p_zero)

19.061783236577952

In [ ]:
cost(proj_mtx)-cost(p_zero)

4.578051641596424

In [ ]:
def training(p_zero,proj_mtx,Xtrain_col,training_label):
 
    p_k = proj_mtx.real
    p_k_1 = p_zero.real
    eta = 0.0001
    while cost(p_k)-cost(p_k_1) > eta:
        y_tr = p_k.transpose().dot(Xtrain_col)
        sp_re = np.zeros(shape=(y_tr.shape[1]-1,y_tr.shape[1]))
        for i in range(y_tr.shape[1]):  
            clf = Lasso(alpha)
            y = y_tr[:,i]
            y_tr = np.delete(y_tr,i,1)
            clf.fit(y_tr,y)
            y_tr=np.insert(y_tr,i,y,axis=1)
            sp_re[:,i] = clf.coef_

      
        num_tr = y_tr.shape[1]

        Ew = np.zeros((pca_components,pca_components))
        Withinclass_error = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
        for i in range(num_tr):
            x = Xtrain_col[:,i]
            label = training_label[i]
            Xtrain_col = np.delete(Xtrain_col,i,1)
            training_label = np.delete(training_label,i)
            Withinclass_error[i,:] = x-np.dot(Xtrain_col,delta(sp_re[:,i],label,training_label))
            Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
            training_label=np.insert(training_label,i,label)

        Ew = np.dot(Withinclass_error.T,Withinclass_error)
        Ew = Ew/number_of_classes*img_in_class

      # cal Eb
        Eb = np.zeros((pca_components,pca_components))
        Betweenclass_error = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
        for i in tqdm(range(num_tr)):
            tmp = np.zeros((pca_components))
            x = Xtrain_col[:,i]
            Xtrain_col = np.delete(Xtrain_col,i,1)
            label = training_label[i]
            other_label_index=(k[label][0])
            for j in range(other_label_index.shape[0]):
                other_label = training_label[j]
                training_label = np.delete(training_label,j)
                tmp += x-(np.dot(Xtrain_col,delta(sp_re[:,j],other_label,training_label)))
                training_label=np.insert(training_label,j,other_label)
            Betweenclass_error[i,:] = tmp
            Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
        Eb = np.dot(Betweenclass_error.T,Betweenclass_error)
        Eb = Eb/(num_data*(number_of_classes-1))
        
        J = np.dot(np.linalg.inv(sw+(beta*Ew)),(sb+(beta*Eb)))
        eigenvalues, eigenvectors, = np.linalg.eig(J)
        eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

        eig_pairs.sort(key=lambda tup:tup[0],reverse=True)
        eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
        eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
        proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()    
        p_k_1 = p_k
        p_k = proj_mtx.real
    return p_k   


In [ ]:
def test(proj_mtx,training_label):
  training_label=training_label.astype(int)
  y_tr = proj_mtx.transpose().dot(Xtrain_col)
  sp_re_test = np.zeros((num_tr,len(Xtest)))
  y_te = proj_mtx.transpose().dot(Xtest_col)

  print('Finding Sparse repre.:....')
  for i in tqdm(range(y_te.shape[1])):    
      clf = Lasso(alpha=0.01)
      y = y_te[:,i]
      clf.fit(y_tr,y)
      sp_re_test[:,i] = clf.coef_


  pred_class_test = np.zeros(y_te.shape[1])
  for i in tqdm(range(y_te.shape[1])):
      y = y_te[:,i]
      label = ytest[i]
      pred_class_test[i] = np.argmin(residu(y,y_tr,sp_re_test[:,i],training_label))

  pred_class_test = pred_class_test.astype(int)
  correct_prediction = 0
  for i in range(y_te.shape[1]):
      if(ytest[i]==pred_class_test[i]):
          correct_prediction=correct_prediction+1

  print('Correct Prediction:',correct_prediction)
  print('Accuracy:',correct_prediction/y_te.shape[1]*100)


In [ ]:
test(proj_mtx,training_label)

In [ ]:
p_k = np.random.rand(pca_components,number_chosen_components)
p_k = training(p_zero,proj_mtx,Xtrain_col,training_label)
test(p_k,training_label)

In [ ]:
cost(p_k)

36.77984276907252

In [ ]:
cost(proj_mtx)

23.639834878174376


# ***Exp-1 divide eigen vector by eigenvalue's square root*** 

In [ ]:
eig_pairs_1 = [(eigenvalues[index], eigenvectors[:,index]/np.sqrt(eigenvalues[index])) for index in range(len(eigenvalues))]

eig_pairs_1.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort_1  = [eig_pairs_1[index][0] for index in range(len(eigenvalues))]
eigvectors_sort_1 = [eig_pairs_1[index][1] for index in range(len(eigenvalues))]
proj_mtx_1 = np.array(eigvectors_sort_1[:number_chosen_components]).transpose()
proj_mtx_1 = proj_mtx_1.real 

In [ ]:
p_k = np.random.rand(pca_components,number_chosen_components)
p_k = training(p_zero,proj_mtx_1,Xtrain_col,training_label)
test(p_k,training_label)

  0%|          | 4/992 [00:00<00:32, 30.21it/s]

Finding Sparse repre.:....


100%|██████████| 992/992 [00:39<00:00, 24.95it/s]

Correct Prediction: 966
Accuracy: 97.37903225806451


# ***EXP-2 Take half eigen vectors***

In [ ]:
proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()
p1 = np.zeros((pca_components,int(number_chosen_components/2)))
index = 0
for i in range(0,number_chosen_components-1,2):
    p1[:,index] = proj_mtx[:,i]+proj_mtx[:,i+1]/2
    index+=1

In [ ]:
test(p1,training_label)

  0%|          | 4/992 [00:00<00:24, 39.65it/s]

Finding Sparse repre.:....


100%|██████████| 992/992 [00:39<00:00, 24.95it/s]

Correct Prediction: 949
Accuracy: 95.66532258064517


# ***EXP 3 Take half eigen vectors from exp 1***

In [ ]:
# proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()
p1_1_2 = np.zeros((pca_components,int(number_chosen_components/2)))
index = 0
for i in range(0,number_chosen_components-1,2):
    p1_1_2[:,index] = proj_mtx_1[:,i]+proj_mtx_1[:,i+1]/2
    index+=1

test(p1_1_2,training_label)

100%|██████████| 992/992 [00:40<00:00, 24.34it/s]

Correct Prediction: 926
Accuracy: 93.34677419354838
Correct Prediction: 926
Accuracy: 93.34677419354838


# ***EXP 3.1 Weighted avarage of eigen vectors from exp 1***

In [ ]:
# proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()
p1_1_3 = np.zeros((pca_components,int(number_chosen_components/2)))
index = 0
for i in range(0,number_chosen_components-1,2):
    p1_1_3[:,index] = (eigenvalues[i]*proj_mtx_1[:,i])+(eigenvalues[i+1]*proj_mtx_1[:,i+1])/(eigenvalues[i]+eigenvalues[i+1])
    index+=1

test(p1_1_3,training_label)

  0%|          | 1/992 [00:00<01:55,  8.59it/s]

Finding Sparse repre.:....
Finding Sparse repre.:....


100%|██████████| 992/992 [00:40<00:00, 24.45it/s]

Correct Prediction: 935
Accuracy: 94.25403225806451
Correct Prediction: 935
Accuracy: 94.25403225806451


# ***EXP 4***

In [ ]:
sp_re = np.zeros(shape=(Xtrain_col.shape[1]-1,Xtrain_col.shape[1]))
for i in range(Xtrain_col.shape[1]):  
    clf = Lasso(alpha)
    y = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    clf.fit(Xtrain_col,y)
    Xtrain_col=np.insert(Xtrain_col,i,y,axis=1)
    sp_re[:,i] = clf.coef_

num_tr = Xtrain_col.shape[1]

Ew_new = np.zeros((pca_components,pca_components))
Withinclass_error_new = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in range(num_tr):
    x = Xtrain_col[:,i]
    label = training_label[i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    training_label = np.delete(training_label,i)
    Withinclass_error_new[i,:] = x-np.dot(Xtrain_col,delta(sp_re[:,i],label,training_label))
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
    training_label=np.insert(training_label,i,label)

Ew_new = np.dot(Withinclass_error_new.T,Withinclass_error_new)
Ew_new = Ew_new/number_of_classes*img_in_class

 
Eb_new = np.zeros((pca_components,pca_components))
Betweenclass_error_new = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in tqdm(range(num_tr)):
    tmp = np.zeros((pca_components))
    x = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    label = training_label[i]
    other_label_index=(k[label][0])
    for j in range(other_label_index.shape[0]):
        other_label = training_label[j]
        training_label = np.delete(training_label,j)
        tmp += x-(np.dot(Xtrain_col,delta(sp_re[:,j],other_label,training_label)))
        training_label=np.insert(training_label,j,other_label)
    Betweenclass_error_new[i,:] = tmp
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
Eb_new = np.dot(Betweenclass_error_new.T,Betweenclass_error_new)
Eb_new = Eb_new/(num_data*(number_of_classes-1))


100%|██████████| 992/992 [21:31<00:00,  1.30s/it]



In [ ]:
# beta = 3
J = np.dot(np.linalg.inv(sw+(beta*Ew_new)),(sb+(beta*Eb_new)))
eigenvalues, eigenvectors, = np.linalg.eig(J)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
p_zero_1 = np.array(eigvectors_sort[:number_chosen_components]).transpose() 

In [ ]:
test(p_zero_1,training_label)

  0%|          | 4/992 [00:00<00:31, 30.98it/s]

Finding Sparse repre.:....


  0%|          | 4/992 [00:00<00:31, 30.98it/s]

Finding Sparse repre.:....


100%|██████████| 992/992 [00:41<00:00, 23.98it/s]

Correct Prediction: 968
Accuracy: 97.58064516129032
Correct Prediction: 968
Accuracy: 97.58064516129032


In [ ]:
p_zero_1=p_zero_1.real
y_tr_init = p_zero_1.transpose().dot(Xtrain_col)
sp_re_init = np.zeros(shape=(y_tr.shape[1]-1,y_tr.shape[1]))
for i in range(y_tr.shape[1]):  
    clf = Lasso(alpha)
    y = y_tr_init[:,i]
    y_tr_init = np.delete(y_tr_init,i,1)
    clf.fit(y_tr_init,y)
    y_tr_init=np.insert(y_tr_init,i,y,axis=1)
    sp_re_init[:,i] = clf.coef_


Ew_1 = np.zeros((pca_components,pca_components))
Withinclass_error_1 = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in range(num_tr):
    x = Xtrain_col[:,i]
    label = training_label[i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    training_label = np.delete(training_label,i)
    Withinclass_error_1[i,:] = x-np.dot(Xtrain_col,delta(sp_re_init[:,i],label,training_label))
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
    training_label=np.insert(training_label,i,label)

Ew_1 = np.dot(Withinclass_error_1.T,Withinclass_error_1)
Ew_1 = Ew_1/number_of_classes*img_in_class


Eb_1 = np.zeros((pca_components,pca_components))
Betweenclass_error_1 = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in tqdm(range(num_tr)):
    tmp = np.zeros((pca_components))
    x = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    label = training_label[i]
    other_label_index=(k[label][0])
    for j in range(other_label_index.shape[0]):
        other_label = training_label[j]
        training_label = np.delete(training_label,j)
        tmp += x-(np.dot(Xtrain_col,delta(sp_re_init[:,j],other_label,training_label)))
        training_label=np.insert(training_label,j,other_label)
    Betweenclass_error_1[i,:] = tmp
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
Eb_1 = np.dot(Betweenclass_error_1.T,Betweenclass_error_1)
Eb_1 = Eb_1/(num_data*(number_of_classes-1))

J = np.dot(np.linalg.inv(sw+(beta*Ew_1)),sb+(beta*Eb_1))
eigenvalues, eigenvectors, = np.linalg.eig(J)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
proj_mtx_new = np.array(eigvectors_sort[:number_chosen_components]).transpose()

100%|██████████| 992/992 [21:30<00:00,  1.30s/it]



In [ ]:
p_k_new = np.random.rand(pca_components,number_chosen_components)
proj_mtx_new = proj_mtx_new.real
p_k_new = training(p_zero_1,proj_mtx_new,Xtrain_col,training_label)

In [ ]:
cost(proj_mtx_new)-cost(p_zero_1)

In [ ]:
cost(p_k)

In [ ]:
test(p_k,training_label)

100%|██████████| 992/992 [00:36<00:00, 27.05it/s]

100%|██████████| 992/992 [00:41<00:00, 24.07it/s]

Correct Prediction: 966
Accuracy: 97.37903225806451
Correct Prediction: 966
Accuracy: 97.37903225806451
